#### Gradio Comparing Transfer Learning Models

In [1]:
!pip install gradio

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [3]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests


# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

mobile_net = tf.keras.applications.MobileNetV2()
inception_net = tf.keras.applications.InceptionV3()



96112376/96112376 [==============================] - 33s 0us/step


In [4]:
def classify_image_with_mobile_net(im):
    im = Image.fromarray(im.astype('uint8'), 'RGB')
    im = im.resize((224, 224))
    arr = np.array(im).reshape((-1, 224, 224, 3))
    arr = tf.keras.applications.mobilenet.preprocess_input(arr)
    prediction = mobile_net.predict(arr).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}
    

In [5]:
def classify_image_with_inception_net(im):
    # Resize the image to
    im = Image.fromarray(im.astype('uint8'), 'RGB')
    im = im.resize((299, 299))
    arr = np.array(im).reshape((-1, 299, 299, 3))
    arr = tf.keras.applications.inception_v3.preprocess_input(arr)
    prediction = inception_net.predict(arr).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}

In [6]:
imagein = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=3)
sample_images = [
                 ["monkey.jpg"],
                 ["sailboat.jpg"],
                 ["bicycle.jpg"],
                 ["download.jpg"],
                 ["fox.jpg"]
]

C:\Users\Sandesh\Anaconda3\lib\site-packages\gradio\inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Sandesh\Anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Sandesh\Anaconda3\lib\site-packages\gradio\outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\Sandesh\Anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [14]:
gr.Interface(
    [classify_image_with_mobile_net, classify_image_with_inception_net],
    imagein,
    label,
    title="MobileNet vs. InceptionNet",
    description="""Let's compare 2 state-of-the-art machine learning models that classify images into one of 1,000 categories: MobileNet (top),
          a lightweight model that has an accuracy of 0.704, vs. InceptionNet
          (bottom), a much heavier model that has an accuracy of 0.779.""",
    examples=sample_images).launch();

DeprecationWarning: The `fn` parameter only accepts a single function, support for a list of functions has been deprecated. Please use gradio.mix.Parallel instead.